# Wikipedia Search
This notebook is to experiment to build search index of Wikipedia and the interfaces to retrieve similar documents to requested text by a user. Here are the traits
* All words and phrases in documents are translated to word embedings using word2vec
* Building binary tree index from document vectors using k-means clustering
* Beam search to traverse the binary tree to retrieve similar documents fast

## import modules

In [30]:
import bz2
import codecs
import copy
from collections import deque
import datetime
import json
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import pandas as pd
import re
import pickle
from sklearn.cluster import KMeans
import sys
import time
import unicodedata
import urllib.request

import MeCab
import pygtrie

from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec

import logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

projectDir = "WikiSearch"

## download wikipedia file

In [2]:
url = "https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2"

compressedFile = os.path.join(projectDir, url.split("/")[-1])

if not os.path.exists(compressedFile):
    start = time.time()
    urllib.request.urlretrieve(url, compressedFile)
    logging.info("completed to download {} (elapsed time:{:.3f})".format(compressedFile, time.time() - start))


## read pages of title and text parts one by one and save to TSV file

**NOTE:** Please download WikiExtractor.py from https://github.com/attardi/wikiextractor and locate it under the same directory

In [3]:
outputDir = os.path.join(projectDir, "dump")
numberOfprocesses = 4
outputFileSize = 1000000000

if not os.path.isdir(outputDir):
    start = time.time()
    os.makedirs(outputDir)
    os.system("WikiExtractor.py -q -o {} --processes {} -b {} {}".format(outputDir, numberOfprocesses, outputFileSize, compressedFile))
    logging.info("completed extract documents by WikiExtractor.py (elapsed time:{:.3f})".format(time.time() - start))


## extract all artiles and output to one TSV file

In [21]:
def extractArticle(lines):
    title = None
    body = None
    if len(lines) > 1:
        title = lines[1]
        body = "".join(lines[2:-1])
        body = body.replace("\n", " ")
        
    return title.strip(), body.strip()

In [5]:
wikiTsvFile = os.path.join(projectDir, "wikiArticle.tsv")

if not os.path.exists(wikiTsvFile):
    start = time.time()
    logging.info("start extracting title and article per line")
    with open(wikiTsvFile, "w", encoding="utf-8") as wf:
        count = 0
        thisDir = os.getcwd()
        for curDir, _, files in os.walk(thisDir):
            for file in files:
                if file.startswith("wiki_"):
                    with codecs.open(os.path.join(curDir,file), "r", "utf-8") as rf:
                        isInPage = False
                        line = rf.readline()

                        while line:
                            if line.startswith("<doc id="):
                                isInPage = True
                                lines = []
                            elif "</doc>" in line:
                                lines.append(line)
                                title, body = extractArticle(lines)
                                if not "(曖昧さ回避)" in title:
                                    wf.write("{}\t{}\n".format(title, body))

                                    count += 1
                                    #if count >= 1500:
                                    #    break

                                    if count % 100000 == 0:
                                        logging.info("processed {:,} documents".format(count))

                                lines = []
                                isInPage = False                            
                                
                            if isInPage:
                                lines.append(line)

                            line = rf.readline()
                        
        logging.info("completed to output {:,} documents to {} (elapsed time:{:.3f})".format(count, wikiTsvFile, time.time() - start))

## tokenize text with MeCab

MeCab is downloaded from https://pypi.org/project/mecab/ and installed

In [6]:
mecab = MeCab.Tagger ("-Ochasen")

def tokenize(text):
    text = unicodedata.normalize("NFKC", text)
    array = [x.split("\t") for x in mecab.parse(text).split("\n") if x.count("\t") == 5]
    return [[x[0] for x in array], [x[3] for x in array]]


## build Trie index for words and phrases
For trie library, pygtrie is used from https://pygtrie.readthedocs.io/en/latest/

In [7]:
class TrieNode:
    def __init__(self, poses):
        self.poses = poses
        self.vector = None
        self.df = 0

def buildTrie(wikiTsvFile):
    start = time.time()
    logging.info("start building trie")
    trie = pygtrie.Trie()
    curDir = os.getcwd()
    with codecs.open(os.path.join(curDir,wikiTsvFile), "r", "utf-8") as f:
        line = f.readline()
        count = 0
        while line:
            title = line.split("\t")[0]
            ret = tokenize(title)
            if len(ret[0]) > 1:
                key = str.join("_", ret[0])
                trie[key] = TrieNode(ret[1])
            count += 1
            if count % 100000 == 0:
                logging.info("processed {:,} titles".format(count))
            line = f.readline()
            
    logging.info("completed to build trie with {:,} titles (elapsed time:{:.3f})".format(count, time.time() - start))
    return trie

def saveTrie(trie, filename):
    start = time.time()
    logging.info("start saving trie file {}".format(filename))
    with open(filename, 'wb') as f:
        pickle.dump(trie, f)
    logging.info("completed to save trie to {}(elapsed time:{:.3f})".format(filename, time.time() - start))
    
def loadTrie(filename):
    start = time.time()
    logging.info("start loading trie file {}".format(filename))
    with open(filename, 'rb') as f:
        trie = pickle.load(f)
    logging.info("completed to load trie file {} (elapsed time:{:.3f})".format(trieWithoutVecFile, time.time() - start))
    return trie


In [8]:
trieWithoutVecFile = os.path.join(projectDir, "trieWithoutVec.pickle")

if not os.path.exists(trieWithoutVecFile):
    trie = buildTrie(wikiTsvFile)
    saveTrie(trie, trieWithoutVecFile)
else:
    trie = loadTrie(trieWithoutVecFile)


2020-09-02 17:03:14,956 : INFO : start loading trie file WikiSearch\trieWithoutVec.pickle
2020-09-02 17:03:46,238 : INFO : completed to load trie file WikiSearch\trieWithoutVec.pickle (elapsed time:31.282)


## tokenize sentenses of Wikipedia descriptions for Word2Vec training
When tokenizing wikipedia titles and the titles consists of multiple words, the words are handled as phrases. In a text, if sequential words matching with one of a phrase are found, POSs of the words are checked to match with the wikipedia title. In case both match, the words are dealt with a phrase

In [9]:
def buildWord2VecTrainFile(trie, wikiTsvFile, sentenceFile):
    start = time.time()
    logging.info("start building Word2Vec train file")
    with codecs.open(wikiTsvFile, "r", "utf-8") as rf, codecs.open(sentenceFile, "w", "utf-8") as wf:
        line = rf.readline()
        count = 0
        while line:
            columns = line.split("\t")
            
            if len(columns) != 2:
                line = rf.readline()
                continue
            
            tokens = tokenize(columns[1])
            sentences = []
            index = 0
            tokenLen = 1
            lastMatchedTokenLen = 1
            lastMatchedSentence = None
            
            while index + tokenLen <= len(tokens[0]):
                sentence = str.join("_", tokens[0][index:index + tokenLen])
                nodeStatus = trie.has_node(sentence)
                
                if nodeStatus & pygtrie.Trie.HAS_VALUE != 0:
                    # check if POSs are equal too
                    if trie[sentence].poses == tokens[1][index:index + tokenLen]:
                        lastMatchedTokenLen = tokenLen
                        lastMatchedSentence = sentence
                    tokenLen += 1
                elif nodeStatus == pygtrie.Trie.HAS_SUBTRIE:
                    tokenLen += 1
                else:
                    if lastMatchedTokenLen == 1:
                        sentences.append(tokens[0][index])
                    else:
                        sentences.append(lastMatchedSentence)
                    index += lastMatchedTokenLen
                    tokenLen = 2
                    lastMatchedTokenLen = 1
            
            if index < len(tokens[0]):
                if lastMatchedTokenLen == 1:
                    sentences.append(tokens[0][index])
                else:
                    sentences.append(str.join("_", tokens[0][index:index + lastMatchedTokenLen]))
            for i in range(index + lastMatchedTokenLen, len(tokens[0])):
                sentences.append(tokens[i])
                
            wf.write("{}\t{}\n".format(columns[0], str.join(" ", sentences)))
            count += 1
            if count % 100000 == 0:
                logging.info("processed {:,} titles".format(count))
            line = rf.readline()
                
    logging.info("completed to build Word2Vec training file {} that consists of {:,} titles (elapsed time:{:.3f})".format(sentenceFile, count, time.time() - start))


In [10]:
tokenSentenceFile = os.path.join(projectDir, "tokenSentence.tsv")

if not os.path.exists(tokenSentenceFile):
    buildWord2VecTrainFile(trie, wikiTsvFile, tokenSentenceFile)

## create corpus file for Word2Vec Training

In [11]:
corpusFile = tokenSentenceFile.replace(".tsv", ".corpus")

if not os.path.exists(corpusFile):
    start = time.time()
    logging.info("start creating corpus file for Word2Vec")
    with codecs.open(tokenSentenceFile, "r", "utf-8") as rf, codecs.open(corpusFile, "w", "utf-8") as wf:
        line = rf.readline()
        count = 0
        while line:
            columns = line.split("\t")
            
            if len(columns) < 2 or not columns[1]:
                line = rf.readline()
                continue
                
            wf.write("{}".format(columns[1]))
            line = rf.readline()
            count += 1

    logging.info("completed to create corpus file for Word2Vec training {} that consists of {:,} articles (elapsed time:{:.3f})".format(corpusFile, count, time.time() - start))



## Word2Vec Training

**References**
* gensim - models.word2vec – Word2vec embeddings
> https://radimrehurek.com/gensim/models/word2vec.html
* Tracking loss and embeddings in Gensim word2vec model
> https://stackoverflow.com/questions/54422810/tracking-loss-and-embeddings-in-gensim-word2vec-model/54423541#54423541

In [12]:
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        self.epoch += 1
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        losses.append(loss_now)
        self.loss_to_be_subed = loss
        logging.info("Loss after epoch {}: {}".format(self.epoch, loss_now))
        model.save("{}.{}.model".format(modelFilePrefix, self.epoch))


# Word2Vec parameters
w2vSize = 400
w2vWindow = 8
w2vMinCount = 10
w2vWorkers = 6
w2vEpochs = 5

losses = []

modelFilePrefix = tokenSentenceFile.replace(".tsv", "") 
modelFile = "{}Model.pickle".format(modelFilePrefix)

start = time.time()
    
if not os.path.exists(modelFile):
    logging.info("start training Word2Vec with {}".format(corpusFile))
    sentences = LineSentence(corpusFile)
    model = Word2Vec(sentences, size=w2vSize, window=w2vWindow, min_count=w2vMinCount, workers=w2vWorkers, iter=w2vEpochs, compute_loss=True, callbacks=[callback()])
    model.save(modelFile)
    plt.plot(losses)
    logging.info("completed to train Word2Vec model (elapsed time:{:.3f})".format(time.time() - start))
else:
    logging.info("start loading Word2Vec model from {}".format(modelFile))
    model = Word2Vec.load(modelFile)
    logging.info("completed to load Word2Vec model (elapsed time:{:.3f})".format(time.time() - start))

2020-09-02 17:03:46,337 : INFO : start loading Word2Vec model from WikiSearch\tokenSentenceModel.pickle
2020-09-02 17:03:46,338 : INFO : loading Word2Vec object from WikiSearch\tokenSentenceModel.pickle
2020-09-02 17:03:54,318 : INFO : loading wv recursively from WikiSearch\tokenSentenceModel.pickle.wv.* with mmap=None
2020-09-02 17:03:54,320 : INFO : loading vectors from WikiSearch\tokenSentenceModel.pickle.wv.vectors.npy with mmap=None
2020-09-02 17:04:02,995 : INFO : setting ignored attribute vectors_norm to None
2020-09-02 17:04:02,997 : INFO : loading vocabulary recursively from WikiSearch\tokenSentenceModel.pickle.vocabulary.* with mmap=None
2020-09-02 17:04:02,997 : INFO : loading trainables recursively from WikiSearch\tokenSentenceModel.pickle.trainables.* with mmap=None
2020-09-02 17:04:02,998 : INFO : loading syn1neg from WikiSearch\tokenSentenceModel.pickle.trainables.syn1neg.npy with mmap=None
2020-09-02 17:04:11,720 : INFO : setting ignored attribute cum_table to None
2020

## save Document count

In [13]:
def saveDocumentCount(file, count):
    start = time.time()
    logging.info("start saving document count to {}".format(file))
    with open(file, 'w') as f:
        f.write(str(count))
    logging.info("completed to save document count file (elapsed time:{:.3f})".format(time.time() - start))
        
def loadDocumentCount(file):
    start = time.time()
    logging.info("start loading document count from {}".format(file))
    with open(file, 'r') as f:
        logging.info("completed to load document count file (elapsed time:{:.3f})".format(time.time() - start)) 
        return int(f.readline())
    

## Word2Vec Trie
extend Trie index with Word2Vec values. Each word has following attributes
* Vector of word
* POSes by MeCab (*in case the word consists of multiple tokens)
* Document frequency

In [14]:
trieW2VFile = os.path.join(projectDir, "trieW2V.pickle")
documentCountFile = os.path.join(projectDir, "documentCount.txt")

if not os.path.exists(trieW2VFile):
    start = time.time()
    logging.info("start fetching vectors to trie")
    trieW2V = pygtrie.Trie()
    count = 0;
    for word in model.wv.vocab.keys():
        vector = np.array(model.wv.word_vec(word))
        if len(vector) == w2vSize and not np.isnan(vector).any():
            poses = trie[word].poses if "_" in word and trie.has_node(word) & pygtrie.Trie.HAS_VALUE != 0 else None
            node = TrieNode(poses)
            node.vector = vector
            trieW2V[word] = node
        
        #print("word:{}, poses:{}, vecvor:{}".format(word, node.poses, node.vector[:5]))
        #if count > 100:
        #    break
            
        count += 1
        if count % 100000 == 0:
            logging.info("processed {:,} words".format(count))
    logging.info("completed to fetch vector to {:,} trie nodes(elapsed time:{:.3f})".format(count, time.time() - start))
    
    start = time.time()
    logging.info("start fetching DF to trie nodes from {}".format(corpusFile))
    with codecs.open(corpusFile, "r", "utf-8") as f:
        line = f.readline()
        documentCount = 0
        while line:
            for word in set(line.split(" ")):
                if trieW2V.has_node(word) & pygtrie.Trie.HAS_VALUE != 0:
                    trieW2V[word].df += 1
            documentCount += 1
            if documentCount % 100000 == 0:
                logging.info("processed {:,} documents".format(documentCount))
            line = f.readline()
    logging.info("completed to fetch DFs to trie nodes with {:,} documents (elapsed time:{:.3f})".format(documentCount, time.time() - start))
    saveTrie(trieW2V, trieW2VFile)
    saveDocumentCount(documentCountFile, documentCount)
else:
    trieW2V = loadTrie(trieW2VFile)
    documentCount = loadDocumentCount(documentCountFile)
    

2020-09-02 17:04:13,790 : INFO : start loading trie file WikiSearch\trieW2V.pickle
2020-09-02 17:04:33,742 : INFO : completed to load trie file WikiSearch\trieWithoutVec.pickle (elapsed time:19.951)
2020-09-02 17:04:33,744 : INFO : start loading document count from WikiSearch\documentCount.txt
2020-09-02 17:04:33,754 : INFO : completed to load document count file (elapsed time:0.010)


## calculate Document Vecotor by BOW

In [15]:
def getDocumentVector(tokens, isIgnorePos = False):
    def initLastMatch():
        return 1, 1, None, None
    
    matchedTokens = []
    missingTokens = []
    vector = np.empty(w2vSize)
    index = 0
    initLastMatch()
    tokenLen, lastMatchedTokenLen, lastMatchedSentence, lastMatchedVector = initLastMatch()
    
    while index + tokenLen <= len(tokens[0]):
        sentence = str.join("_", tokens[0][index:index + tokenLen])
        nodeStatus = trieW2V.has_node(sentence)
        if isIgnorePos:
            if nodeStatus & pygtrie.Trie.HAS_VALUE != 0:
                matchedTokens.append(sentence)
                node = trieW2V[sentence]
                vector += node.vector * np.log(float(documentCount) / node.df)
            else:
                missingTokens.append(tokens[0][index])
            index += 1
        else:
            if nodeStatus & pygtrie.Trie.HAS_VALUE != 0:
                if tokenLen == 1 or trieW2V[sentence].poses == tokens[1][index:index + tokenLen]:
                    node = trieW2V[sentence]
                    lastMatchedTokenLen = tokenLen
                    lastMatchedSentence = sentence
                    lastMatchedVector = node.vector * np.log(float(documentCount) / node.df)
                tokenLen += 1
            elif nodeStatus == pygtrie.Trie.HAS_SUBTRIE:
                tokenLen += 1
            else:
                if lastMatchedSentence:
                    matchedTokens.append(lastMatchedSentence)
                    vector += lastMatchedVector
                    index += lastMatchedTokenLen
                else:
                    missingTokens.append(tokens[0][index])
                    index += 1
                tokenLen, lastMatchedTokenLen, lastMatchedSentence, lastMatchedVector = initLastMatch()

    # L2 regularization
    vector /= np.linalg.norm(vector, ord=2)
    return (vector, matchedTokens, missingTokens)

In [16]:
# if a document consists of limited number of words, or lots of missing words in Word2Vec Trie index, process of the document is skipped
minTokenCount = 30
maxMissingTokenRatio = 0.1

docVectorFile = os.path.join(projectDir, 'docVector.pickle')
start = time.time()

if not os.path.exists(docVectorFile):
    logging.info("start calculating document vector")
    docVectorMatrix = []
    with codecs.open(tokenSentenceFile, "r", "utf-8") as f:
        line = f.readline()
        count = 0
        while line:
            columns = line.strip().split('\t')
            if len(columns) == 2 and columns[0] and columns[1]:
                tokens = [columns[1].split(" "), [None] * len(columns[1])]
                vector, matchedTokens, missingTokens = getDocumentVector(tokens, True)
                tokenCount = len(matchedTokens) + len(missingTokens)
                if tokenCount >= minTokenCount:
                    maxMissingTokenCount = int(math.ceil(tokenCount * maxMissingTokenRatio))
                    if len(missingTokens) < maxMissingTokenCount and not np.isnan(vector).any():
                        buf = [columns[0]]
                        buf.extend(vector.tolist())
                        docVectorMatrix.append(buf)                        
                        count += 1
                        
                        if count % 100000 == 0:
                            logging.info("processed {:,} documents".format(count))
                            
            line = f.readline()
        
        docVectorDf = pd.DataFrame(docVectorMatrix)
        logging.info("completed to fetch DFs to trie nodes with {:,} documents (elapsed time:{:.3f})".format(count, time.time() - start))
        with open(docVectorFile, 'wb') as f:
            pickle.dump(docVectorDf, f)
            
else:
    logging.info("start loading document vector files {}".format(docVectorFile))
    with open(docVectorFile, 'rb') as f:
        docVectorDf = pickle.load(f)
    logging.info("completed to load {:,} documents from files {}(elapsed time:{:.3f})".format(len(docVectorDf), docVectorFile, time.time() - start))


2020-09-02 17:04:33,802 : INFO : start loading document vector files WikiSearch\docVector.pickle
2020-09-02 17:05:07,340 : INFO : completed to load 1,031,231 documents from files WikiSearch\docVector.pickle(elapsed time:33.539)


## k-means clustering for documents by document vectors
k-means clustering documents to 2 clusters, and repeat it for each cluster iteratively until number of documents is less than or equal to minLeavesCount(=100 in this case) The result is used to build binary tree. 2 centroids of a cluster become the edges, and final clusterIds are index of the leaf nodes.

**NOTE:** This part is very time consuming. On my machine (Xeon E5-1620 3.5GHz, RAM 32GB, Win10 x64, using 6 cores) it tooks about 7 days for the completion. This should be rewritten using GPU in future

In [17]:
docVectorKmeansFile = os.path.join(projectDir, 'docVectorKmeans.pickle')
docVectorKmeansFile_ = os.path.join(projectDir, 'docVectorKmeans.pickle_')
centroidsFile = os.path.join(projectDir, 'centroids.pickle')
centroidsFile_ = os.path.join(projectDir, 'centroids.pickle_')

if not os.path.exists(docVectorKmeansFile):
    start = time.time()

    maxIterationCount = 100
    minLeavesCount = 100
    nodeIdColumnIndex = w2vSize + 1

    docVectorDf[nodeIdColumnIndex] = 0
    parentNodeIdQueue = deque([0])
    centroids = {}

    kmeans = KMeans(n_clusters=2, max_iter=maxIterationCount, n_jobs=w2vWorkers, verbose=1)

    while parentNodeIdQueue:
        parentNodeId = parentNodeIdQueue.popleft()
        startNodeId = parentNodeId * 2 + 1
        features = docVectorDf[docVectorDf[:][nodeIdColumnIndex] == parentNodeId]
        features = features.iloc[:, 1:w2vSize+1]
        kmeans_model = kmeans.fit(features)
        predict = kmeans.predict(features) + parentNodeId * 2 + 1
        features[nodeIdColumnIndex] = predict

        for i, centroid in enumerate(kmeans_model.cluster_centers_):
            centroids[startNodeId + i] = centroid.tolist()

        for tuple in features.itertuples():
            docVectorDf.iloc[tuple[0], nodeIdColumnIndex] = tuple[-1]

        for id in range(startNodeId, startNodeId + 2):
            if len(docVectorDf[docVectorDf[:][nodeIdColumnIndex] == id]) > minLeavesCount:
                parentNodeIdQueue.append(id)

        logging.info("k-means clustering has been done for nodeId {} with {:,} documents".format(parentNodeId, len(features)))
        with open(docVectorKmeansFile_, 'wb') as f:
            pickle.dump(docVectorDf, f)
        with open(centroidsFile_, 'wb') as f:
            pickle.dump(centroids, f)

    os.rename(docVectorKmeansFile_, docVectorKmeansFile)
    os.rename(centroidsFile_, centroidsFile)

    logging.info("completed k-means clustering for {:,} documents(elapsed time:{:.3f})".format(len(docVectorDf), time.time() - start))
    
else:
    logging.info("start loading {} and {}".format(docVectorKmeansFile, centroidsFile))
    with open(docVectorKmeansFile, 'rb') as f:
        docVectorDf = pickle.load(f)
    with open(centroidsFile, 'rb') as f:
        centroids = pickle.load(f)
    logging.info("completed to load {} and {}(elapsed time:{:.3f})".format(docVectorKmeansFile, centroidsFile, time.time() - start))
    

2020-09-02 17:05:07,368 : INFO : start loading WikiSearch\docVectorKmeans.pickle and WikiSearch\centroids.pickle
2020-09-02 17:05:28,614 : INFO : completed to load WikiSearch\docVectorKmeans.pickle and WikiSearch\centroids.pickle(elapsed time:54.814)


## implement beam search for clustered documents
steps
1. tokenize requested text and translate to vector
2. from top node start traversing binary tree with limited BFS, calculating cosine similarity between the text vector and centroids
3. When completing traversing, get clusterIds of the leaf nodes and aggregate all of the associated documents
4. Calculate cosine similarity between user requested text and wikipedia document one by one to come up with the most similar wikipedia documents.

In [24]:
class Node:
    def __init__(self, clusterId):
        self.clusterId = clusterId
        self.similarity = 0.
        self.isLeaf = False
        
    def getLeftClusterId(self):
        return 2 * self.clusterId + 1
    
    def getRightClusterId(self):
        return 2 * (self.clusterId + 1)

class SimilarDocument:
    def __init__(self, clusterId, title, similarity):
        self.clusterId = clusterId
        self.title = title
        self.similarity = similarity
    
class NodeQueue:
    def __init__(self, docVectorDf, centroids, maxBreadth):
        self.docVectorDf = docVectorDf
        self.clusterIdSet = set(docVectorDf[:][w2vSize + 1].unique())
        self.centroids = centroids
        self.maxBreadth = maxBreadth
        self.queue = []
    
    def getNodes(self):
        return [(i, n) for i, n in enumerate(self.queue) if not n.isLeaf]

    def getAllLeaves(self):
        return sorted([n for n in self.queue if n.isLeaf], key=lambda n: n.similarity, reverse=True)
    
    def getSimilarDocuments(self, docVector):
        similarDocuments = []
        conditions = None
        for leaf in self.getAllLeaves():
            if conditions is None:
                conditions = self.docVectorDf[:][w2vSize + 1] == leaf.clusterId
            else:
                conditions |= self.docVectorDf[:][w2vSize + 1] == leaf.clusterId
        for tuple in self.docVectorDf[conditions].itertuples():
            clusterId = tuple[0]
            title = tuple[1]
            similarity = np.dot(docVector, tuple[2:w2vSize + 2])
            similarDocuments.append(SimilarDocument(clusterId, title, similarity))
        return sorted(similarDocuments, key=lambda d: d.similarity, reverse=True)
                                    
    def hasNode(self):
        return len(self.getNodes()) > 0
    
    def popLeft(self):
        node = None
        nodeTuples = self.getNodes()
        if nodeTuples:
            node = nodeTuples[0][1]
            del self.queue[nodeTuples[0][0]]
        return node
    
    def append(self, docVector, node):
        node.isLeaf = node.clusterId in self.clusterIdSet
        node.similarity = np.dot(docVector, self.centroids[node.clusterId])
        self.queue.append(node)
        while len(self.queue) > self.maxBreadth:
            minSimilarity = 1.
            minIndex = -1
            for i, n in enumerate(self.queue):
                if n.similarity < minSimilarity:
                    minSimilarity = n.similarity
                    minIndex = i
            del self.queue[minIndex]
            
    def clear(self):
        self.queue.clear()
        

In [25]:
def getSimilarDocuments(nodeQueue, docVector):
    nodeQueue.clear()
    nodeQueue.append(docVector, Node(1))
    nodeQueue.append(docVector, Node(2))
    while nodeQueue.hasNode():
        node = nodeQueue.popLeft()
        nodeQueue.append(docVector, Node(node.getLeftClusterId()))
        nodeQueue.append(docVector, Node(node.getRightClusterId()))
    return nodeQueue.getSimilarDocuments(docVector)
            

In [33]:
def getTopXSimilarDocuments(nodeQueue, topX, text):
    start = datetime.datetime.now()
    tokens = tokenize(text)
    vector, matchedTokens, missingTokens = getDocumentVector(tokens)
    documents = []
    tokenCount = len(matchedTokens) + len(missingTokens)
    missingTokenRatio = len(missingTokens) / tokenCount
    docs = []
    if missingTokenRatio < maxMissingTokenRatio:
        docs = getSimilarDocuments(nodeQueue, vector)
        for doc in docs[:topX]:
            documents.append({'title':doc.title, 'url':'https://ja.wikipedia.org/wiki/{}'.format(urllib.parse.quote(doc.title)), 'similarity':doc.similarity})
    end = datetime.datetime.now()
    response = {
        'request':
        {
            'requestTime':start.strftime('%Y-%m-%dT%H:%M:%S'),
            'text':text,
            'tokenCount':tokenCount,
            'missingTokenRatio':missingTokenRatio,
            'missingTokens':missingTokens
        },
        'response':
        {
            'responseTime':end.strftime('%Y-%m-%dT%H:%M:%S'),
            'documentCount':len(docs),
            'documents':documents,
        },
        'elapsedMilliseconds':int(round((end - start).total_seconds(), 4) * 1000)
    }
    return json.dumps(response, ensure_ascii=False)


## Test

In [38]:
text = '地球温暖化により台風の勢力が拡大している懸念について'

nodeQueue = NodeQueue(docVectorDf, centroids, 10)

getTopXSimilarDocuments(nodeQueue, 10, text)

'{"request": {"requestTime": "2020-09-02T17:59:41", "text": "地球温暖化により台風の勢力が拡大している懸念について", "tokenCount": 10, "missingTokenRatio": 0.0, "missingTokens": []}, "response": {"responseTime": "2020-09-02T17:59:41", "documentCount": 653, "documents": [{"title": "台風", "url": "https://ja.wikipedia.org/wiki/%E5%8F%B0%E9%A2%A8", "similarity": 0.6928481710987686}, {"title": "2015年", "url": "https://ja.wikipedia.org/wiki/2015%E5%B9%B4", "similarity": 0.6775010330686655}, {"title": "複雑な動きをする台風", "url": "https://ja.wikipedia.org/wiki/%E8%A4%87%E9%9B%91%E3%81%AA%E5%8B%95%E3%81%8D%E3%82%92%E3%81%99%E3%82%8B%E5%8F%B0%E9%A2%A8", "similarity": 0.6726890610325177}, {"title": "斜面崩壊", "url": "https://ja.wikipedia.org/wiki/%E6%96%9C%E9%9D%A2%E5%B4%A9%E5%A3%8A", "similarity": 0.6506094006276739}, {"title": "ハリケーン・カタリーナ", "url": "https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%AA%E3%82%B1%E3%83%BC%E3%83%B3%E3%83%BB%E3%82%AB%E3%82%BF%E3%83%AA%E3%83%BC%E3%83%8A", "similarity": 0.6463698809338628}, {"title": "沖縄台風", 